# CIFAR10 + Transfer Learning Tutorial

## Goals

CIFAR 10 is a dataset of color images that have 10 classifications. We will be looking at different ways of creating a model for this dataset. The goals for this week are to:

1. Build your own CNN for CIFAR10.
2. Explore transfer learning and its uses (Still finishing this up).
3. Explore the following links to think about tasks for our project moving forward. (http://cs231n.stanford.edu/reports/2016/pdfs/214_Report.pdf), (https://medium.com/@coviu/how-we-used-ai-to-translate-sign-language-in-real-time-782238ed6bf), (https://blogs.nvidia.com/blog/2017/05/11/ai-translates-sign-language/).

The model for CIFAR 10 will take a much longer time to train. This is because the CIFAR10 database is much larger (more images + images are bigger) and the model is more complex. Try to budget time for running the model, but if you cannot finish training the model/the results aren't great that is okay. 

## Resources

Really good notes if you want to get a better understanding of the topics (http://cs231n.github.io/).

## Import Libraries

In [ ]:
# Libraries for building network
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Libraries for dataset
import torchvision
import torchvision.transforms as transforms

# Miscellaneous Libraries
import time

## Define Network

In [ ]:
class tutorial_model(nn.Module):
    def __init__(self):
        """ Initialize all layers of model """
        # TODO                                                 
        
    def forward(self, x):
        """ Chain all layers together """
        # TODO

## Set Hyperparameters

In [ ]:
# Number of epochs for training
num_epochs =      

# Batch Size for training/testing
batch_size =   

# Learning Rate for optimizer
learning_rate = 

# Dimensions of CIFAR10
dim =     

## Setup Data Loader

In [ ]:
# Transformation for training data
transform_train = torchvision.transforms.Compose([
    transforms.RandomResizedCrop(dim, scale=(0.7, 1.0), ratio=(1.0,1.0)),
    transforms.ColorJitter(
            brightness=0.1,
            contrast=0.1,
            saturation=0.1,
            hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize((0.5,), (0.5,)),             # Normalize grayscale data
])

# Transformation for training data
transform_test = transforms.Compose([
    transforms.CenterCrop(dim),
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize((0.5,), (0.5,)),             # Normalize grayscale data
])

In [ ]:
# Download training data
trainset = torchvision.datasets.CIFAR10(root='./files', train=True, download=True, 
                                      transform=transform_train)

# Initialize dataloader for training data
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                                           num_workers=8)

# Download testing Data
testset = torchvision.datasets.CIFAR10(root='./files', train=False, download=False, 
                                     transform=transform_test)

# Initialize dataloader for testing data
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, 
                                          num_workers=8)

### CIFAR10 Dataset
![](https://alexisbcook.github.io/assets/cifar10.png)

## Initialize Model/Optimizer

In [ ]:
# Initialize previously defined model
model = tutorial_model()                                               

# Definie loss function (Cross Entropy Loss)
criterion = nn.CrossEntropyLoss()                                      

# Initialize Optimizer (ADAM)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)     

# Set model to training (updating weights)
model.train();                                                        

## Train Model

In [ ]:
# Store time to calculate train time
start_time = time.time()

# Store loss and accuracy data
loss = []
accuracy = []

# Train the model
# Loop for number of epochs
for epoch in range(num_epochs):
    # Loop through data in batch sized increments
    for batch_idx, (X_train_batch, Y_train_batch) in enumerate(train_loader):
        # If trained on all data in epoch, move onto next epoch
        if(Y_train_batch.shape[0]<batch_size):
            continue

        # Forward pass through network
        output = model(X_train_batch)                           
        # Calculate loss of predictions
        curr_loss = criterion(output, Y_train_batch)            
        # Store loss
        loss.append(curr_loss.item())                           

        # Clear last calculation
        optimizer.zero_grad()                                   
        # Calculate gradient based on loss
        curr_loss.backward()                                    
        # Update model weights
        optimizer.step()                                        

        # Extract model predictions
        _, predicted = torch.max(output.data, 1) 
        # Calculate number of correct predictions
        correct = (predicted == Y_train_batch).sum().item()     
        # Calculate/store accuracy
        accuracy.append(correct/Y_train_batch.size(0))          
        
        # Intermitently print statistics
        if batch_idx % 100 == 0:
            print('Epoch: ' + str(epoch+1) + '/' + str(num_epochs) + ', Step: ' 
                  + str(batch_idx+1) + '/' + str(len(train_loader)) + ', Loss: ' 
                  + str(curr_loss.item()) + ', Accuracy: ' 
                  + str(correct/Y_train_batch.size(0)*100) + '%')

# Store time to calculate train time
end_time = time.time()

# Print train time
print('Run Time: ' + str(end_time - start_time))

## Test Model

In [ ]:
# Test the model
# Set model to testing (constant weights)
model.eval()

with torch.no_grad():
    # Store number of correct/total samples in test data
    correct = 0
    total = 0
    
    # Loop through test data
    for X_test_batch, Y_test_batch in test_loader:
        # Forward pass through network
        output = model(X_test_batch)  
        
        # Extract prediction
        _, predicted = torch.max(output.data, 1)    
        
        # Update total number of sample
        total += Y_test_batch.size(0)  
        
        # Update number of correct predictions
        correct += (predicted == Y_test_batch).sum().item()     

print('Test Accuracy: ' + str((correct/total) * 100) + '%')

## Transfer Learning